In [92]:
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
matplotlib.use("pgf")
plt.rcParams.update({
    "text.usetex": True,
    "pgf.rcfonts": False,
    "pgf.texsystem": "pdflatex",
})
from urllib.parse import urlparse
from scipy.stats import pearsonr
from scipy import stats

# Load data

In [70]:
all_poms_before_sampling = pd.read_csv("data/all_poms_before_sampling.csv")
all_repos_before_sampling = pd.read_csv("data/all_repos_before_sampling.csv")

C:\Users\jacob\AppData\Local\Temp\ipykernel_14416\2805030735.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  all_poms_before_sampling = pd.read_csv("data/all_poms_before_sampling.csv")
C:\Users\jacob\AppData\Local\Temp\ipykernel_14416\2805030735.py:2: DtypeWarning: Columns (5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  all_repos_before_sampling = pd.read_csv("data/all_repos_before_sampling.csv")


## Random Sampling and removing duplicates

In [71]:
all_poms = all_poms_before_sampling.copy()
l_all_poms = len(all_poms)

random_seed = 42
np.random.seed(random_seed)

# Shuffle the DataFrame randomly
all_poms = all_poms.sample(frac=1).reset_index(drop=True)

# Remove duplicates from poms to only mantain one version of each package
all_poms = all_poms.drop_duplicates(subset=['groupId', 'artifactId'])

# Reset the index of the deduplicated DataFrame
all_poms.reset_index(drop=True, inplace=True)

print("all_poms: " + str(l_all_poms) + " -> " + str(len(all_poms)))

# only take rows that are later than or in 2016 
all_poms['releaseDate'] = pd.to_datetime(all_poms['releaseDate'])
all_poms = all_poms[all_poms['releaseDate'].dt.year >= 2016]

# Save the DataFrame to a CSV file
all_poms.to_csv('data/all_poms.csv', index=False)

all_poms: 934266 -> 199188


In [72]:
all_repos = all_repos_before_sampling.copy()
l_all_repos = len(all_repos)

# Drop duplicates from repos
all_repos = all_repos.drop_duplicates(subset=['id', 'pomId', 'name', 'url'])
all_repos = all_repos[all_repos['pomId'].isin(all_poms['id'])]

print("all_repos: " + str(l_all_repos) + " -> " + str(len(all_repos)))

# Save the DataFrame to a CSV file
all_repos.to_csv('data/all_repos.csv', index=False)

all_repos: 157387 -> 16781


In [73]:
# <erge poms and repos and name the id of left id_pom and id_repo
all_poms_repos = pd.merge(all_poms, all_repos, left_on='id', right_on='pomId', how='inner')
all_poms_repos = all_poms_repos.rename(columns={'id_x': 'id_pom', 'id_y': 'id_repo'})
all_poms_repos = all_poms_repos.rename(columns={'url_x': 'url_pom', 'url_y': 'url_repo'})

print("all_poms_repos: " + str(len(all_poms_repos)))

all_poms_repos: 16781


# Showing the distribution of the data

In [98]:
df = all_poms.copy()
df['releaseDate'] = pd.to_datetime(df['releaseDate'])

# Group by year and count the rows
yearly_counts_v2 = df.groupby(df['releaseDate'].dt.year).size().reset_index(name='count')

# Display the result
print(yearly_counts_v2)

plt.figure(figsize=(3.5, 3))
ax = sns.barplot(x="releaseDate", y="count", data=yearly_counts_v2, palette='viridis')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right', fontsize=8)
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x/1000)))
plt.xlabel('Year', fontsize=8)
plt.ylabel('Number of Packages (x1000)', fontsize=8)
plt.savefig('graphs/packages_per_year.pgf')
plt.close()

   releaseDate  count
0         2016  23701
1         2017  21406
2         2018  24571
3         2019  31089
4         2020  19561
5         2021  21559
6         2022  23481
7         2023  33815


# Average packages with at least 1 repository per year

In [106]:
df = all_poms.copy()
df['releaseDate'] = pd.to_datetime(all_poms['releaseDate'])

def calc_at_least_one_repo_percentage_over_time(repos, period='Y'):
    # Filter the DataFrame
    poms_with_repos = repos[repos['repositoriesAmount'] >= 1]

    # Resample the data by the desired time period and calculate the percentages
    percentages = poms_with_repos.resample(period, on='releaseDate')['id'].nunique() / repos.resample(period, on='releaseDate')['id'].nunique() * 100

    return percentages

# Calculate the percentages over time 
percentages = calc_at_least_one_repo_percentage_over_time(df)

# Convert the index to a numerical format (e.g., number of days since the start of the period)
x = np.array(range(len(percentages.index)))

# Convert the percentages to a numpy array
y = percentages.values

# Create a mask where both x and y are not nan
mask = ~np.isnan(y)
x = x[mask]
y = y[mask]
percentages = percentages[mask]

plt.figure(figsize=(3.5, 3))
plt.plot(percentages.index.year, percentages, label='Combined', color='#492c68', marker='o')
plt.xticks(percentages.index.year, fontsize=8)
plt.xticks(rotation=45, horizontalalignment='right')
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.xlabel('Year', fontsize=8)
plt.ylabel('Percentage of packages with at least 1 repository (%)', fontsize=8)
plt.savefig('graphs/packages_with_repositories_percentages.pgf')
plt.close()

# Find pearson correlation between percentage and time
corr, p_val = pearsonr(x, y)
print('Pearsons correlation: %.3f' % corr, "p-value: ", p_val)

Pearsons correlation: -0.791 p-value:  0.01948714873964627


In [76]:
# Focusing on the range of 'repositoriesAmount' from 1 to 10
data_repos = all_poms[(all_poms['repositoriesAmount'] >= 1) & (all_poms['repositoriesAmount'] <=10)].copy()

# Calculating the percentage of packages for each repository amount
repos_counts = data_repos['repositoriesAmount'].value_counts()
total_packages = len(all_poms)
percentage_repos = (repos_counts / total_packages) * 100

# Sorting the index for plotting
percentage_repos = percentage_repos.sort_index()

# Plotting
plt.figure(figsize=(3.5, 3))
plt.subplots_adjust(bottom=0.2)
bars = sns.barplot(x=percentage_repos.index, y=percentage_repos.values, palette='viridis')
plt.ylim(0, 1.2 * max(percentage_repos.values))
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)

for bar in bars.patches:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 0.05, f'{yval:.2f}%', ha='center', va='bottom', fontsize=6)

plt.xlabel('Number of Repositories Specified', fontsize=8)
plt.ylabel('Percentage of Packages', fontsize=8)
plt.xticks(range(10))  
ax = plt.gca() 
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: int(x + 1)))
plt.savefig('graphs/repositories_count.pgf')
plt.close()

In [77]:
def calc_at_least_one_repo_percentage(repos):
    # Filter the DataFrame
    poms_with_repos = repos[repos['repositoriesAmount'] >= 1]
    
    # Count the number of unique IDs with at least 1 repository
    unique_ids_with_repos = poms_with_repos['id'].nunique()
    
    # Count the total number of unique IDs
    total_unique_ids = repos['id'].nunique()
    
    # Calculate the percentage
    percentage = (unique_ids_with_repos / total_unique_ids) * 100
    
    print(f'Percentage of unique IDs with at least 1 repository: {percentage}%')
    

calc_at_least_one_repo_percentage(all_poms)

Percentage of unique IDs with at least 1 repository: 4.079163382417174%


# Do repository ids commonly change when new versions of the same package are released?

In [78]:
def analyze_id_variations(repos):
    # Group by groupId and artifactId
    grouped = repos.groupby(['groupId', 'artifactId', 'url_y'])
    changes_count = 0
    total_groups = 0

    for _, group in grouped:
        total_groups += 1
        # Check if there are variations in ID across different versions
        if group['id_y'].nunique() > 1:
            changes_count += 1

    # Calculate the percentage of groups where IDs change
    if total_groups > 0:
        change_percentage = (changes_count / total_groups) * 100
    else:
        change_percentage = 0

    return change_percentage

merged_df = pd.merge(all_poms_before_sampling, all_repos_before_sampling, left_on='id', right_on='pomId', how='inner')

# Call the function and print the result
percentage_changes = analyze_id_variations(merged_df)
print(f"Percentage of groups with ID changes across versions: {percentage_changes}%")

Percentage of groups with ID changes across versions: 0.6806385270786407%


In [79]:
maven_errors = [
    {"issue": "Forbidden", "error_message": "Forbidden (403)"},
    {"issue": "Blocked Mirror", "error_message": "Blocked mirror"},
    {"issue": "Not found in Central", "error_message": "Could not find"},
    {"issue": "Non-resolvable parent POM", "error_message": "Non-resolvable parent POM"},
    {"issue": "Does not exist", "error_message": "does not exist"},
    {"issue": "Unknown packaging", "error_message": " Unknown packaging"},
    {"issue": "Connection refused", "error_message": "Connection refused"} 
]


def match_error_messages(dataset_msgs, predefined_errors):
    # Initialize a dictionary to count occurrences of each issue
    issue_counts = {error['issue']: 0 for error in predefined_errors}

    for dataset_msg in dataset_msgs:
        for error in predefined_errors:
            if error['error_message'].lower() in dataset_msg.lower():
                issue_counts[error['issue']] += 1

    return issue_counts


In [80]:
error_all_poms = all_poms.copy()
error_all_poms['year'] = pd.to_datetime(error_all_poms['releaseDate']).dt.year
error_all_poms.dropna(subset=['mavenErrorMessage'], inplace=True)

print(len(error_all_poms))
grouped =  error_all_poms.groupby('year')


# calculate percentage of error types for each year
issues_per_year = {}

for year, group in grouped:
    error_messages = group['mavenErrorMessage'].dropna()
    issue_counts = match_error_messages(error_messages, maven_errors)
    total_errors = sum(issue_counts.values())
    
    for issue in issue_counts:
        issue_counts[issue] = issue_counts[issue] / total_errors * 100 if total_errors > 0 else 0
    issues_per_year[year] = issue_counts
    
plot_issues = ['Connection refused']

# plot the results
plt.figure(figsize=(3.5, 3))

for issue in plot_issues:
    years = list(issues_per_year.keys())
    percentages = [issues_per_year[year][issue] for year in years]
    plt.plot(years, percentages, marker='o', color='#492c68', label=issue)   
    
plt.xticks(years, fontsize=8)
plt.xticks(rotation=45, horizontalalignment='right')
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.xlabel('Year', fontsize=8)
plt.ylabel('Percentage of Connection Refused Error', fontsize=8)
plt.savefig('graphs/connection_refused.pgf')
plt.close()


1741


In [105]:
# Calculate the percentage of packages that have an error
total_packages = len(all_poms)
poms_with_errors = all_poms.dropna(subset=['mavenErrorMessage'], inplace=False)
packages_with_errors = len(poms_with_errors)
percentage_with_errors = (packages_with_errors / total_packages) * 100

print(f"Percentage of packages that have an error: {percentage_with_errors}%")

# Filter the DataFrame to include only rows where repositoriesAmount is greater than or equal to 1
packages_with_repos = all_poms[all_poms['repositoriesAmount'] >= 1].copy()

# Count the number of packages that have one or more repositories
num_packages_with_repos = len(packages_with_repos)

# Filter the DataFrame to include only rows where mavenErrorMessage is not null
packages_with_repos_and_errors = packages_with_repos.dropna(subset=['mavenErrorMessage'])

# Count the number of packages that have one or more repositories and have an error
num_packages_with_repos_and_errors = len(packages_with_repos_and_errors)

percentage_packages_with_repos_with_error =  num_packages_with_repos_and_errors / num_packages_with_repos * 100

print(f"Percentage of packages with repositories that have an error: {percentage_packages_with_repos_with_error}%")

# Filter the DataFrame to include only rows where repositoriesAmount is 0
packages_with_no_repos = all_poms[all_poms['repositoriesAmount'] == 0].copy()

# Filter the DataFrame to include only rows where repositoriesAmount is greater than or equal to 1
packages_with_repos = all_poms[all_poms['repositoriesAmount'] >= 1].copy()

# Calculate the percentage of errors for packages with no repositories
packages_with_no_repos['has_error'] = packages_with_no_repos['mavenErrorMessage'].notna()
percentage_errors_no_repos = packages_with_no_repos['has_error'].mean()
print(f"Percentage of packages without repositories that have an error: {percentage_errors_no_repos*100}%")

# Calculate the percentage of errors for packages with repositories
packages_with_repos['has_error'] = packages_with_repos['mavenErrorMessage'].notna()
percentage_errors_with_repos = packages_with_repos['has_error'].mean()

# Perform the t-test
t_stat, p_val = stats.ttest_ind(packages_with_no_repos['has_error'], packages_with_repos['has_error'])

print(f"T-statistic: {t_stat}")
print(f"P-value: {p_val}")

Percentage of packages that have an error: 0.8740705783124062%
Percentage of packages with repositories that have an error: 0.8861538461538463%
Percentage of packages without repositories that have an error: 0.8735704378320379%
T-statistic: -0.1193421947180227
P-value: 0.9050043761588642


In [89]:
# Check for every row in in all_poms_repos whether the url_repo is in the error message
def check_url_in_error(row):
    if pd.isnull(row['mavenErrorMessage']):
        return np.nan
    elif row['url_repo'] in row['mavenErrorMessage']:
        return row['mavenErrorMessage']
    else:
        return np.nan

url_poms_repos = all_poms_repos.copy()
urls_with_errors = url_poms_repos.dropna(subset=['mavenErrorMessage', 'url_repo'], inplace=False).copy()
urls_with_errors['error_message_with_url'] = urls_with_errors.apply(check_url_in_error, axis=1)

# Percentage of errors containing repo url 
packages_with_url_as_error = urls_with_errors.dropna(subset=['error_message_with_url'], inplace=False)
deduplicated_packages = packages_with_url_as_error.drop_duplicates(subset=['groupId', 'artifactId', 'version'])

percentage_packages_with_url_as_error = len(deduplicated_packages) / len(all_poms_repos) * 100
print(f"Percentage of packages with repository URL in error message: {percentage_packages_with_url_as_error}%")

Percentage of packages with repository URL in error message: 0.15493713127942316%


# Testing availability of repository URLs

In [94]:
import requests

# Initialize a cache dictionary to store URL status codes
url_cache = {}

# Function to check URL availability and use cache
def check_url_availability(url):
    if url in url_cache:
        return url_cache[url]
    try:
        response = requests.get(url, timeout=2)
        status_code = response.status_code
        url_cache[url] = status_code  # Cache the result
        return status_code
    except requests.exceptions.RequestException:
        return None


In [96]:
# Strip right of / and get unique urls
all_poms_repos_copy = all_poms_repos.copy()
all_poms_repos_copy['url_repo'] = all_poms_repos_copy['url_repo'].str.rstrip('/')
unique_urls_repos = all_poms_repos_copy.drop_duplicates(subset=['url_repo'], keep='first').copy()

# Apply the function to each row and create a new 'stillAvailable' column
unique_urls_repos['stillAvailable'] = unique_urls_repos['url_repo'].apply(check_url_availability)

# Classify NaN values as timeouts
unique_urls_repos['stillAvailable'] = unique_urls_repos['stillAvailable'].fillna('Timeout')

# Calculate the percentage of 200 status codes
total_percentage_200 = (unique_urls_repos['stillAvailable'] == 200).mean() * 100

# Calculate the percentage of other status codes
total_percentage_other = 100 - total_percentage_200

# Plotting
plt.figure(figsize=(3, 2.5))
plt.pie([total_percentage_200, total_percentage_other], labels=['Reachable', 'Unreachable'], autopct='%1.1f%%', startangle=90, colors=['#74758d', '#738292'], textprops={'fontsize': 10})
plt.subplots_adjust(left=0.2, right=0.8)
plt.savefig('graphs/unreachable_urls.pgf')
plt.close()

# Step 5: Print the percentages
print(f'Total Percentage of 200 Status Codes: {total_percentage_200:.1f}%')
print(f'Total Percentage of Other Status Codes: {total_percentage_other:.1f}%')

Total Percentage of 200 Status Codes: 0.0%
Total Percentage of Other Status Codes: 100.0%


In [97]:
# Merge the dataframes on 'url_repo'
all_poms_repos_copy = all_poms_repos.copy()
merged_df = pd.merge(all_poms_repos_copy, unique_urls_repos[['url_repo', 'stillAvailable']], on='url_repo', how='left')

# Convert the 'releaseDate' column to datetime
merged_df['releaseDate'] = pd.to_datetime(merged_df['releaseDate'])

# Extract the year from 'releaseDate'
merged_df['year'] = merged_df['releaseDate'].dt.year

# Group by year
grouped = merged_df.groupby('year')

# Initialize a dictionary to store the percentage of unreachable URLs per year
unreachable_percentage_per_year = {}

# Calculate the percentage of unreachable URLs per year
for year, group in grouped:
    total_urls = len(group)
    unreachable_urls = len(group[group['stillAvailable'] != 200])
    unreachable_percentage = (unreachable_urls / total_urls) * 100 if total_urls > 0 else 0
    unreachable_percentage_per_year[year] = unreachable_percentage


# Plotting
plt.figure(figsize=(3.5, 3))
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.xticks(list(unreachable_percentage_per_year.keys()), fontsize=8)
plt.xticks(rotation=45, horizontalalignment='right')
plt.plot(list(unreachable_percentage_per_year.keys()), list(unreachable_percentage_per_year.values()), marker='o', color='#492c68',)
plt.xlabel('Year', fontsize=8)
plt.ylabel('Percentage of Unreachable URLs (%)', fontsize=8)
plt.savefig('graphs/unreachable_urls_per_year.pgf')
plt.close()

## Http vs Https

In [90]:
updated_poms_repos_df = all_poms_repos
updated_poms_repos_df['releaseYear'] = pd.to_datetime(updated_poms_repos_df['releaseDate']).dt.year

# Filtering data for the years 2015 to 2023
filtered_years_df = updated_poms_repos_df[(updated_poms_repos_df['releaseYear'] >= 2016) & (updated_poms_repos_df['releaseYear'] <= 2023)]

# Categorizing the URL type with the adjusted criteria, handling NaN or non-string values
filtered_years_df['url_type'] = filtered_years_df['url_repo'].apply(
    lambda x: 'https' if isinstance(x, str) and 'https://' in x else ('http' if isinstance(x, str) and 'http://' in x else 'other')
)

# Counting the usage of HTTP, HTTPS, and other URLs per year
http_https_other_counts = filtered_years_df.groupby(['releaseYear', 'url_type']).size().unstack(fill_value=0)

# Converting counts to percentages of the total for each year
http_https_other_percentages = http_https_other_counts.div(http_https_other_counts.sum(axis=1), axis=0) * 100

# Remove other from plot (these were urls that that did not mention http or https protocol)
http_https_other_percentages = http_https_other_percentages.drop(columns=['other'])

# Plotting
plt.figure(figsize=(3.5, 3))
plt.plot(http_https_other_percentages.index, http_https_other_percentages['http'], marker='o', color='#492e6b', label='HTTP')
plt.plot(http_https_other_percentages.index, http_https_other_percentages['https'], marker='o', color='#405f82', label='HTTPS')
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.xlabel('Year', fontsize=8)
plt.ylabel('Percentage of Total URLs', fontsize=8)
plt.xticks(http_https_other_percentages.index, rotation=45)
plt.legend(title='URL Type')
plt.savefig('graphs/http_vs_https.pgf')
plt.close()

# Repository URL and Id Analysis

In [91]:
# Grouping by year and id_repo, counting unique URLs per id_repo
poms_repos_df = all_poms_repos
poms_repos_df['url_repo'] = poms_repos_df['url_repo'].str.rstrip('/')
poms_repos_df['releaseYear'] = pd.to_datetime(poms_repos_df['releaseDate']).dt.year
unique_urls_per_id_year = poms_repos_df.groupby(['releaseYear', 'id_repo'])['url_repo'].nunique()

# Finding the ids with at least two different urls (collisions)
collision_ids_per_year = unique_urls_per_id_year[unique_urls_per_id_year > 1]

# Counting total ids and collision ids per year
total_ids_per_year = poms_repos_df.groupby('releaseYear')['id_repo'].nunique()
collision_count_per_year = collision_ids_per_year.groupby(level=0).count()

# Calculating the percentage of ids with collisions per year
collision_percentage_per_year = (collision_count_per_year / total_ids_per_year) * 100

# Plotting
plt.figure(figsize=(3.5, 3))
plt.subplots_adjust(bottom=0.2)
bars = sns.barplot(x=collision_percentage_per_year.index, y=collision_percentage_per_year.values, palette='viridis')

for bar in bars.patches:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 0.5, f'{yval:.2f}%', ha='center', va='bottom', fontsize=6)
plt.ylim(0, 1.2 * max(collision_percentage_per_year.values))
plt.xticks(rotation=45, horizontalalignment='right')
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.xlabel('Year', fontsize=8)
plt.ylabel('Percentage of Collisions', fontsize=8)
plt.savefig('graphs/id_collisions.pgf')
plt.close()

print(f'Average percentage of collisions: {collision_percentage_per_year.mean():.2f}%')

correlation, p_value = pearsonr(collision_percentage_per_year.index, collision_percentage_per_year.values)
print(f'Correlation: {correlation:.4f}' + '\n' + f'p-value: {p_value:.3f}')

Average percentage of collisions: 14.76%
Correlation: 0.6405
p-value: 0.087
